In [13]:
pip install aiohttp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.6/401.6 kB 906.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.3 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install aiohttp nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio
from tqdm.asyncio import tqdm
from datetime import datetime, timedelta

nest_asyncio.apply()

API_KEY = 'NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91'
BASE_URL_MARKET_CAP = 'https://financialmodelingprep.com/api/v3/historical-market-capitalization/'
BASE_URL_PRICE = 'https://financialmodelingprep.com/api/v3/historical-price-full/'

# Definir el ticker específico
ticker = 'AAPL'

async def fetch(session, url, retries=5):
    for attempt in range(retries):
        async with session.get(url) as response:
            if response.status == 200:
                try:
                    data = await response.json()
                    if not data:
                        print(f"No data returned for URL: {url}")
                    return data
                except Exception as e:
                    print(f"Failed to parse JSON from {url}: {e}")
                    return None
            elif response.status == 429:
                retry_after = int(response.headers.get("Retry-After", 5))
                if attempt == 0:  # Print message only once
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                await asyncio.sleep(retry_after)
            else:
                print(f"Failed request to {url}: {response.status}")
                await asyncio.sleep(2 ** attempt)
    return None

async def get_market_cap(session, ticker, start_date, end_date):
    url = f"{BASE_URL_MARKET_CAP}{ticker}?from={start_date}&to={end_date}&apikey={API_KEY}"
    return await fetch(session, url)

async def get_price(session, ticker, date):
    url = f"{BASE_URL_PRICE}{ticker}?from={date}&to={date}&serietype=line&apikey={API_KEY}"
    return await fetch(session, url)

async def find_last_available_date(data, date_key='date'):
    """Find the last available date in the data."""
    if not data:
        return None
    dates = [entry[date_key] for entry in data]
    dates.sort(reverse=True)
    return dates[0] if dates else None

async def process_ticker_year(session, ticker, year):
    start_date = f"{year}-12-01"
    end_date = f"{year}-12-31"
    
    market_cap_data = await get_market_cap(session, ticker, start_date, end_date)
    if market_cap_data:
        print(f"Market cap data for {ticker} in {year}: {market_cap_data}")
        last_date = await find_last_available_date(market_cap_data)
        if last_date:
            last_entry = next((item for item in market_cap_data if item["date"] == last_date), None)
            if not last_entry:
                print(f"No market cap data for {ticker} on {last_date}")
                return None
            market_cap = last_entry['marketCap']
            
            price_data = await get_price(session, ticker, last_date)
            if price_data and 'historical' in price_data:
                print(f"Price data for {ticker} on {last_date}: {price_data['historical']}")
                last_price_entry = next((item for item in price_data['historical'] if item["date"] == last_date), None)
                if not last_price_entry:
                    print(f"No price data for {ticker} on {last_date}")
                    return None
                price = last_price_entry['close']
                
                outstanding_shares = market_cap / price
                return [ticker, year, last_date, market_cap, price, outstanding_shares]
            else:
                print(f"No price data for {ticker} on {last_date}")
        else:
            print(f"No last available date found for {ticker} between {start_date} and {end_date}")
    else:
        print(f"No market cap data for {ticker} between {start_date} and {end_date}")
    return None

async def main():
    results = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for year in range(1985, 2023 + 1):
            tasks.append(process_ticker_year(session, ticker, year))
        
        for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing tickers"):
            result = await task
            if result:
                results.append(result)
    
    # Guardar los resultados en un archivo CSV
    if results:
        market_cap_df = pd.DataFrame(results, columns=['Ticker', 'Year', 'Date', 'MarketCap', 'Price', 'OutstandingShares'])
        market_cap_df.to_csv('market_cap_shares_AAPL.csv', index=False)
    else:
        print("No results to save")

# Ejecutar la función principal
asyncio.run(main())


Processing tickers:   0%|          | 0/39 [00:01<?, ?it/s]

Market cap data for AAPL in 2008: [{'symbol': 'AAPL', 'date': '2008-12-31', 'marketCap': 76106772000}, {'symbol': 'AAPL', 'date': '2008-12-30', 'marketCap': 76679606080}, {'symbol': 'AAPL', 'date': '2008-12-29', 'marketCap': 76928565840}, {'symbol': 'AAPL', 'date': '2008-12-26', 'marketCap': 76181686560}, {'symbol': 'AAPL', 'date': '2008-12-24', 'marketCap': 75683767040}, {'symbol': 'AAPL', 'date': '2008-12-23', 'marketCap': 76928565840}, {'symbol': 'AAPL', 'date': '2008-12-22', 'marketCap': 76181686560}, {'symbol': 'AAPL', 'date': '2008-12-19', 'marketCap': 79916082960}, {'symbol': 'AAPL', 'date': '2008-12-18', 'marketCap': 79418163440}, {'symbol': 'AAPL', 'date': '2008-12-17', 'marketCap': 79169203680}, {'symbol': 'AAPL', 'date': '2008-12-16', 'marketCap': 84895278160}, {'symbol': 'AAPL', 'date': '2008-12-15', 'marketCap': 84148398880}, {'symbol': 'AAPL', 'date': '2008-12-12', 'marketCap': 87384875760}, {'symbol': 'AAPL', 'date': '2008-12-11', 'marketCap': 84397358640}, {'symbol': 'A

KeyError: 'open'

In [25]:
# Leer y mostrar el archivo CSV generado
market_cap_df = pd.read_csv('market_cap_shares_AAPL.csv')

# Mostrar las primeras filas del DataFrame para asegurarnos de que se ha leído correctamente
print(market_cap_df.head())

# Mostrar información general sobre el DataFrame
print(market_cap_df.info())

# Resumen estadístico de las columnas numéricas
print(market_cap_df.describe())

  Ticker  Year        Date     MarketCap     Price  OutstandingShares
0   AAPL  1986  1986-12-31    2578219571   0.18080       1.426006e+10
1   AAPL  2015  2015-12-31  580554031680  26.32000       2.205752e+10
2   AAPL  2017  2017-12-29  865303303480  42.31000       2.045151e+10
3   AAPL  1992  1992-12-31    7239041922   0.53348       1.356947e+10
4   AAPL  1995  1995-12-29    3920458348   0.28460       1.377533e+10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ticker             39 non-null     object 
 1   Year               39 non-null     int64  
 2   Date               39 non-null     object 
 3   MarketCap          39 non-null     int64  
 4   Price              39 non-null     float64
 5   OutstandingShares  39 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.0+ KB
None
              Year     Mark

In [26]:
market_cap_df

,Ticker,Year,Date,MarketCap,Price,OutstandingShares
0,AAPL,1986,1986-12-31,2578219571,0.180800,1.426006e+10
1,AAPL,2015,2015-12-31,580554031680,26.320000,2.205752e+10
2,AAPL,2017,2017-12-29,865303303480,42.310000,2.045151e+10
3,AAPL,1992,1992-12-31,7239041922,0.533480,1.356947e+10
4,AAPL,1995,1995-12-29,3920458348,0.284600,1.377533e+10
5,AAPL,2014,2014-12-31,639403657640,27.600000,2.316680e+10
6,AAPL,2009,2009-12-31,191347420320,7.530000,2.541134e+10
7,AAPL,2022,2022-12-30,2064941499390,129.930000,1.589272e+10
8,AAPL,1988,1988-12-30,5133838176,0.359380,1.428526e+10
9,AAPL,2005,2005-12-30,59783000760,2.570000,2.326187e+10


In [41]:
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio
from tqdm import tqdm
from datetime import datetime, timedelta

nest_asyncio.apply()

API_KEY = 'NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91'
BASE_URL_MARKET_CAP = 'https://financialmodelingprep.com/api/v3/historical-market-capitalization/'
BASE_URL_PRICE = 'https://financialmodelingprep.com/api/v3/historical-price-full/'

# Cargar los tickers del archivo CSV
tickers_df = pd.read_csv('unique_sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()  # Only take the first 10 tickers

semaphore = asyncio.Semaphore(10)  # Limit concurrency to 10 requests

async def fetch(session, url, retries=5, timeout=60):
    for attempt in range(retries):
        try:
            async with session.get(url, timeout=timeout) as response:
                if response.status == 200:
                    try:
                        data = await response.json()
                        return data if data else None
                    except Exception as e:
                        return None
                elif response.status == 429:
                    retry_after = int(response.headers.get("Retry-After", 5))
                    await asyncio.sleep(retry_after)
                else:
                    await asyncio.sleep(2 ** attempt)
        except asyncio.TimeoutError:
            await asyncio.sleep(2 ** attempt)
    return None

async def get_market_cap(session, ticker, start_date, end_date):
    url = f"{BASE_URL_MARKET_CAP}{ticker}?from={start_date}&to={end_date}&apikey={API_KEY}"
    return await fetch(session, url)

async def get_price(session, ticker, date):
    url = f"{BASE_URL_PRICE}{ticker}?from={date}&to={date}&serietype=line&apikey={API_KEY}"
    return await fetch(session, url)

async def find_last_available_date(data, date_key='date'):
    """Find the last available date in the data."""
    if not data:
        return None
    dates = [entry[date_key] for entry in data]
    dates.sort(reverse=True)
    return dates[0] if dates else None

async def process_ticker_year(session, ticker, year):
    start_date = f"{year}-12-01"
    end_date = f"{year}-12-31"
    
    market_cap_data = await get_market_cap(session, ticker, start_date, end_date)
    if market_cap_data:
        last_date = await find_last_available_date(market_cap_data)
        if last_date:
            last_entry = next((item for item in market_cap_data if item["date"] == last_date), None)
            if not last_entry:
                return None
            market_cap = last_entry['marketCap']
            
            price_data = await get_price(session, ticker, last_date)
            if price_data and 'historical' in price_data:
                last_price_entry = next((item for item in price_data['historical'] if item["date"] == last_date), None)
                if not last_price_entry:
                    return None
                price = last_price_entry['close']
                
                outstanding_shares = market_cap / price
                return [ticker, year, last_date, market_cap, price, outstanding_shares]
    return None

async def process_ticker(session, ticker):
    async with semaphore:
        results = []
        successful_years = 0
        years_with_data = []
        for year in range(1985, 2023 + 1):
            result = await process_ticker_year(session, ticker, year)
            if result:
                results.append(result)
                successful_years += 1
                years_with_data.append(year)
        await asyncio.sleep(1)  # Reduced sleep time to improve performance
        print(f"{ticker}: {successful_years} years of data found ({', '.join(map(str, years_with_data))})")
        return results

async def main():
    all_results = []
    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=120)) as session:
        tasks = [process_ticker(session, ticker) for ticker in tickers]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing tickers"):
            ticker_results = await f
            all_results.extend(ticker_results)
    
    # Guardar los resultados en un archivo CSV
    if all_results:
        market_cap_df = pd.DataFrame(all_results, columns=['Ticker', 'Year', 'Date', 'MarketCap', 'Price', 'OutstandingShares'])
        market_cap_df.to_csv('market_cap_shares.csv', index=False)
    else:
        print("No results to save")

# Ejecutar la función principal
if __name__ == "__main__":
    asyncio.run(main())

# Leer y mostrar el archivo CSV generado
market_cap_df = pd.read_csv('market_cap_shares.csv')

# Mostrar las primeras filas del DataFrame para asegurarnos de que se ha leído correctamente
print(market_cap_df.head())

# Mostrar información general sobre el DataFrame
print(market_cap_df.info())

# Resumen estadístico de las columnas numéricas
print(market_cap_df.describe())


Processing tickers:   0%|          | 1/826 [00:07<1:41:49,  7.41s/it]

KRFT: 0 years of data found ()


Processing tickers:   0%|          | 2/826 [00:09<55:54,  4.07s/it]  

NOVL: 0 years of data found ()


Processing tickers:   0%|          | 3/826 [00:10<36:35,  2.67s/it]

...

M: 32 years of data found (1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)


Processing tickers: 100%|█████████▉| 825/826 [1:05:11<00:01,  1.24s/it]

PG: 38 years of data found (1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)
AMT: 26 years of data found (1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)


Processing tickers: 100%|██████████| 826/826 [1:05:16<00:00,  4.74s/it]

PPG: 39 years of data found (1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023)
  Ticker  Year        Date     MarketCap   Price  OutstandingShares
0   CDAY  2018  2018-12-31  4.833748e+09   34.49       1.401493e+08
1   CDAY  2019  2019-12-31  9.818525e+09   67.88       1.446453e+08
2   CDAY  2020  2020-12-31  1.584718e+10  106.56       1.487160e+08
3   CDAY  2021  2021-12-31  1.589089e+10  104.46       1.521242e+08
4   CDAY  2022  2022-12-30  9.858575e+09   64.15       1.536800e+08
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17991 entries, 0 to 17990
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ticker             17991 non-null  object 
 1   Year               17991 non-null  int64  
 2   Date               17991